In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import tempfile
import shutil
import matplotlib.pyplot as plt

# print the tensor flow version
print(tf.__version__)

In [ ]:
from utils import display_errors
from utils import unfold_labels
from dataset_helper import DataSetHelper

In [ ]:
num_classes = 10
learning_rate = 0.01
batch_size = 100

In [ ]:
# setup tensorboard
tensorboard_file_name = "./TensorFlowBoard/tensor_flow_ccn"
shutil.rmtree(tensorboard_file_name, ignore_errors = True) # start fresh each time
writer = tf.summary.FileWriter(tensorboard_file_name)

In [ ]:
#load mnist data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

training_data_set = DataSetHelper(x_train, y_train, one_hot = True, num_classes = num_classes, rescale = True)
test_data_set = DataSetHelper(x_test, y_test, one_hot = True, num_classes = num_classes, rescale = True)

In [ ]:
training_set_size = x_train.shape[0]
epochs = training_set_size * 3

In [ ]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [ ]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, 784], name = "x")

x_image = tf.reshape(x, [-1, 28, 28, 1])

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32, name = "keep_prob")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

z = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

#W = tf.Variable(tf.random.uniform([784, 10]), name = "W")
#b = tf.Variable(tf.random.uniform([10]), name = "b")
#z = tf.add(tf.matmul(x, W), b, name = "z")

y = tf.nn.softmax(z, name = "y")
y_ = tf.placeholder(tf.float32, [None, 10], name = "y_")

In [ ]:
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=y))

train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

# calculate the accuracy on the training set
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1), name = "correct_prediction")
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name = "accuracy")

prediction = tf.argmax(y,1)

In [ ]:
loss_batch = []
accuracy_batch = []

In [ ]:
with tf.Session() as sess:
    # write the graph to the tensorboard log
    writer.add_graph(sess.graph)
    
    tf.global_variables_initializer().run()

    for index in range(500):
        batch_xs, batch_ys = training_data_set.next_batch(batch_size)
        
        if index % (epochs/100) == 0:
            train_loss = loss.eval(feed_dict={x: batch_xs, y_: batch_ys, keep_prob: 1.0})
            train_accuracy = accuracy.eval(feed_dict={x: batch_xs, y_: batch_ys, keep_prob: 1.0})
            loss_batch.append(train_loss)
            accuracy_batch.append(train_accuracy)
        
        if index % (epochs/10) == 0:
            print("Batch:", index, "; Accuracy: ", train_accuracy, "; Loss: ", train_loss)
        
        sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys, keep_prob: 1.0})
    
    pred = prediction.eval(feed_dict={x: test_data_set.images, keep_prob: 1.0}, session=sess)
    print("Accuracy: ", sess.run(accuracy, feed_dict={x: test_data_set.images, y_: test_data_set.labels, keep_prob: 1.0}))

In [ ]:
#with tf.Session() as sess:
#    sess.run(tf.global_variables_initializer())
#    for i in range(10):
#        batch = mnist.train.next_batch(50)
#        if i % 100 == 0:
#          train_accuracy = accuracy.eval(feed_dict={
#              x: batch[0], y_: batch[1], keep_prob: 1.0})
#          print('step %d, training accuracy %g' % (i, train_accuracy))
#        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

#    prediction = tf.argmax(y_conv,1)
#    pred = prediction.eval(feed_dict={x: mnist.test.images}, session=sess)
    
#    print('test accuracy %g' % accuracy.eval(feed_dict={
#          x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

In [ ]:
plt.plot(range(0, len(loss_batch)), loss_batch, 'r-', label='Batch Loss')
plt.plot(range(0, len(accuracy_batch)), accuracy_batch, 'b-', label='Batch Accuracy')
plt.legend(loc='upper right')
plt.title('Training accuracy and loss')
plt.show()

In [ ]:
labels = test_data_set.labels.argmax(axis=1)
conf = tf.confusion_matrix(labels, pred, 10)
with tf.Session() as sess:
    print(conf.eval(session=sess))

In [ ]:
display_errors(x_test, y_test, pred, grid = 8)